In [1]:
import pandas as pd

In [34]:
df = pd.DataFrame()

In [35]:
df['ID_pedido'] = [1, 2, 3, 4, 5]
df['Produtos'] = [
    ["Halteres", "Tapete", "Rolo"],
    ["Bandas", "Garrafa", "Tapete"],
    ["Creatina", "Bolsa", "Proteína"],
    ["Cordas", "Luvas", "Bolsa"],
    ["Halteres", "Elíptico", "Creatina"]
]

In [36]:
df

,ID_pedido,Produtos
0,1,"[Halteres, Tapete, Rolo]"
1,2,"[Bandas, Garrafa, Tapete]"
2,3,"[Creatina, Bolsa, Proteína]"
3,4,"[Cordas, Luvas, Bolsa]"
4,5,"[Halteres, Elíptico, Creatina]"


In [37]:
# Criando um DataFrame para o one-hot encoding
produtos_unicos = set(produto for sublist in df['Produtos'] for produto in sublist)
produtos_unicos = sorted(produtos_unicos)  # Ordenar para consistência